In [22]:
import os

from formulaic.utils.null_handling import drop_rows

instances = []
with open('..'+os.sep+'tests'+os.sep+'tests2Run.txt') as file:
    for line in file:
        if line[0] != '#' and line[0] != '\n':
            if '\n' in line:
                line = line[:-1]
            tokens = line.split('\t')
            instance = tokens[0]
            instances.append(instance)

for instance in instances:
    file = '..'+os.sep+'outputs'+os.sep+instance+'_statistics_.txt'
    with open(file) as f:
        print(file,f.read())


../outputs/test_exp_3_statistics_.txt total jobs;total completed;total expired
2498;1916;582


In [23]:
import pandas as pd
from pymongo import MongoClient

# Connect to the MongoDB server
client = MongoClient('mongodb://localhost:27017/')
db = client['EUA_db']
collection = db['events_data']

# Retrieve data from the collection
data = list(collection.find())

# Filter events for jobs
job_events = [event for event in data if event['status'] in ['TypeEvent.user_arrival', 'TypeEvent.user_completed', 'TypeEvent.user_expired']]

# Create jobs DataFrame
jobs = pd.DataFrame(job_events)
jobs=jobs[['origin_id','time','status']]


jobs['id'] = 0
for indx,row in jobs.iterrows():
    msg = row['origin_id']
    jobs.loc[indx,'id'] = msg.split(' ')[1]
    
# display(jobs.head())


# Assuming `jobs` DataFrame is already created and contains the necessary data

# Group by 'id' and sort by 'time' in reverse order
jobs_sorted = jobs.sort_values(by=['id', 'time'], ascending=[True, False])

# Initialize a list to store the results
time_differences = []

# Iterate over each group
for _, group in jobs_sorted.groupby('id'):
    # Find the arrival time
    arrival_times = group[group['status'] == 'TypeEvent.user_arrival']['time'].values
    if len(arrival_times) == 0:
        continue
    
    
    # Find the expired/completed time
    arrival_time = float(arrival_times[0])
    # print(group)
    for _, row in group.iterrows():
        if row['status'] in ['TypeEvent.user_expired', 'TypeEvent.user_completed']:
            
            time_diff = float(row['time']) - arrival_time
            time_differences.append({
                'id': row['id'],
                'time_difference': time_diff
            })

# Convert the results to a DataFrame
time_diff_df = pd.DataFrame(time_differences)

# Display the result
display(time_diff_df)
time_diff_df.describe()

/tmp/ipykernel_44507/428678321.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  jobs.loc[indx,'id'] = msg.split(' ')[1]


,id,time_difference
0,0,17.0
1,1,18.0
2,10,20.0
3,100,20.0
4,1000,15.7
...,...,...
4409,997,1.0
4410,998,4.0
4411,998,4.0
4412,999,10.9


,time_difference
count,4414.000000
mean,7.167694
std,4.553873
min,1.000000
25%,4.000000
50%,7.000000
75%,9.700000
max,22.700000


In [24]:
import pandas as pd
from pymongo import MongoClient

# Connect to the MongoDB server
client = MongoClient('mongodb://localhost:27017/')
db = client['EUA_db']
collection = db['events_data']

# Retrieve data from the collection
data = list(collection.find())

# Filter events for jobs
servers_events = [event for event in data if event['status'] in ['TypeEvent.server_switched_off', 'TypeEvent.server_switched_on']]

# Create jobs DataFrame
servers = pd.DataFrame(servers_events)
servers=servers[['origin_id','time','status']]


servers['id'] = 0
servers['type server'] = ''
for indx,row in servers.iterrows():
    msg = row['origin_id']
    servers.loc[indx,'id'] = msg.split(' ')[1]
    servers.loc[indx,'type server'] = msg.split(' ')[2]
    
display(servers)


# Assuming `jobs` DataFrame is already created and contains the necessary data

# Group by 'id' and sort by 'time' in reverse order
servers_sorted = servers.sort_values(by=['id', 'time'], ascending=[True, False])

# Initialize a list to store the results
time_differences = []

# Iterate over each group
for _, group in servers_sorted.groupby('id'):
    # Find the arrival time
    arrival_times = group[group['status'] == 'TypeEvent.server_switched_on']['time'].values
    if len(arrival_times) == 0:
        continue
    
    # Find the expired/completed time
    i=0
    for _, row in group.iterrows():
        if row['status'] in ['TypeEvent.server_switched_off']:
            arrival_time = float(arrival_times[i])        
            time_diff = float(row['time']) - arrival_time
            time_differences.append({
                'id': row['id'],
                'time_difference': time_diff,
                'type_server': row['type server']
            })
            i += 1

# Convert the results to a DataFrame
time_diff_df = pd.DataFrame(time_differences)
#display(time_diff_df)

summed_time_df = time_diff_df.groupby(['id','type_server']).sum('time_difference').reset_index()

# Rename the column for clarity
summed_time_df.rename(columns={'time_difference': 'total_time'}, inplace=True)

# Display the result
display(summed_time_df)

/tmp/ipykernel_44507/1464136998.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '18' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  servers.loc[indx,'id'] = msg.split(' ')[1]


,origin_id,time,status,id,type server
0,Server 18 L,39,TypeEvent.server_switched_on,18,L
1,Server 11 L,40,TypeEvent.server_switched_on,11,L
2,Server 18 L,40,TypeEvent.server_switched_off,18,L
3,Server 11 L,41,TypeEvent.server_switched_off,11,L
4,Server 11 L,43,TypeEvent.server_switched_on,11,L
...,...,...,...,...,...
405,Server 29 L,1398,TypeEvent.server_switched_on,29,L
406,Server 3 L,1399,TypeEvent.server_switched_off,3,L
407,Server 29 L,1400,TypeEvent.server_switched_off,29,L
408,Server 3 L,1401,TypeEvent.server_switched_on,3,L


,id,type_server,total_time
0,0,L,224.0
1,1,L,494.0
2,10,L,376.0
3,11,L,325.0
4,12,L,454.0
5,13,L,345.0
6,14,L,316.0
7,15,L,406.0
8,16,L,486.0
9,17,L,438.0


In [7]:
import pandas as pd
from pymongo import MongoClient

# Connect to the MongoDB server
client = MongoClient('mongodb://localhost:27017/')
db = client['EUA_db']
collection = db['events_data']

# Retrieve data from the collection
data = list(collection.find())

# Filter events for jobs
job_events = [event for event in data if event['status'] in ['TypeEvent.user_arrival','TypeEvent.user_expired', 'TypeEvent.user_completed','TypeEvent.user_rejected_all_servers_out','TypeEvent.user_rejected_all_servers_off','TypeEvent.user_rejected_not_enough_time','TypeEvent.user_interrupted','TypeEvent.user_server_assigned','TypeEvent.server_going_to_process_user','TypeEvent.user_lock_for_resources','TypeEvent.user_unlock_for_resources','TypeEvent.user_rejected_not_enough_resources']]

# Create jobs DataFrame
jobs = pd.DataFrame(job_events)
jobs = jobs[['origin_id','time','status']]

jobs['id'] = -1
jobs['comp'] = -1
jobs["due"] = -1
for indx,row in jobs.iterrows():
    msg = row['origin_id']
    jobs.loc[indx,'id'] = msg.split(' ')[1]
    jobs.loc[indx,'comp'] = msg.split(' ')[4]
    jobs.loc[indx,'due'] = msg.split(' ')[5]

# Group by 'id' and sort by 'time' in reverse order
jobs['id']=jobs['id'].astype(int)
jobs['time'] = jobs['time'].astype(float)
jobs_sorted = jobs.sort_values(by=['id', 'time'], ascending=[True,True])

# Iterate over each group
event_jobs = []
for i, group in jobs_sorted.groupby('id'):
    event_jobs.append({'comp': [str(group['comp'].values[0])],'due': [str(group['due'].values[0])]})
    for _, row in group.iterrows():
        event = row['status'].replace('TypeEvent.','')
        if event not in event_jobs[-1]:
            event_jobs[-1][event] = []
        event_jobs[-1][event].append(str(row['time']))
        
    print(event_jobs[-1])

for job in event_jobs:
    for event in job:
        if len(job[event]) > 1:
            job[event] = ",".join(job[event])
        else:
            job[event] = str(job[event][0])

df = pd.DataFrame(event_jobs)

try:
    df = df[['comp', 'due', 'user_completed','user_arrival',
       'user_expired', 'user_server_assigned', 'user_lock_for_resources',
       'user_unlock_for_resources','user_rejected_not_enough_resources', 'user_rejected_not_enough_time','user_rejected_all_servers_out', 'user_interrupted']]
except Exception as e:
    print(e)

df.to_csv("events_jobs.csv")
# time_diff_df = pd.DataFrame(time_differences)
# 
# # Display the result
# display(time_diff_df)
# time_diff_df.describe()

/tmp/ipykernel_9497/2144648546.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  jobs.loc[indx,'id'] = msg.split(' ')[1]
/tmp/ipykernel_9497/2144648546.py:25: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '6' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  jobs.loc[indx,'comp'] = msg.split(' ')[4]
/tmp/ipykernel_9497/2144648546.py:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '17' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  jobs.loc[indx,'due'] = msg.split(' ')[5]


{'comp': ['6'], 'due': ['17'], 'user_arrival': ['1.0'], 'user_rejected_all_servers_out': ['1.0', '3.0', '4.0', '6.0', '8.0', '11.0', '12.0', '14.0', '15.0', '18.0'], 'user_expired': ['18.0']}
{'comp': ['5'], 'due': ['96'], 'user_arrival': ['78.0'], 'user_rejected_all_servers_out': ['78.0', '80.0', '82.0', '83.0', '85.0', '87.0', '90.0', '93.0', '95.0', '96.0'], 'user_expired': ['96.0']}
{'comp': ['9'], 'due': ['206'], 'user_arrival': ['190.0'], 'user_server_assigned': ['190.0'], 'user_lock_for_resources': ['190.0'], 'user_unlock_for_resources': ['190.0'], 'user_completed': ['199.0', '199.0']}
{'comp': ['8'], 'due': ['263'], 'user_arrival': ['249.0'], 'user_server_assigned': ['249.0'], 'user_lock_for_resources': ['249.0'], 'user_unlock_for_resources': ['249.0'], 'user_completed': ['257.0', '257.0']}
{'comp': ['9'], 'due': ['269'], 'user_arrival': ['249.0'], 'user_rejected_all_servers_out': ['249.0', '251.0', '254.0', '256.0', '257.0', '258.0', '261.0', '264.0', '265.0', '266.0', '267.0'

In [26]:
print(df.columns)

Index(['comp', 'due', 'user_completed', 'user_arrival', 'user_expired',
       'user_server_assigned', 'user_lock_for_resources',
       'user_unlock_for_resources', 'user_rejected_not_enough_resources',
       'user_rejected_not_enough_time', 'user_rejected_all_servers_out',
       'user_interrupted'],
      dtype='object')
